In [0]:
%sql
CREATE TABLE IF NOT EXISTS workspace.gold_staging.stg_methods
USING DELTA
AS
SELECT
  time,
  methods.sop,
  methods.id AS method_id,
  methods.meth_id,
  methods.temp_id,
  methods.report_template_version,
  methods.seq_id,
  methods.seq_version
FROM workspace.silver_schema.silver_delta_table;

MERGE INTO workspace.gold_staging.stg_methods AS target  -- Target Delta table alias
USING (
  SELECT * FROM (
    SELECT
      time,                                             -- Select time column from source
      methods.sop,                                      -- Select sop from nested methods struct
      methods.id AS method_id,                          -- Alias methods.id as method_id (used as key)
      methods.meth_id,                                  -- Select meth_id
      methods.temp_id,                                  -- Select temp_id
      methods.report_template_version,                  -- Select report_template_version
      methods.seq_id,                                   -- Select seq_id
      methods.seq_version,                              -- Select seq_version
      ROW_NUMBER() OVER (                               -- Add row number to deduplicate source
        PARTITION BY time, methods.id                   -- Partition by composite key to group duplicates
        ORDER BY time                                   -- Order within each group (can be any deterministic order)
      ) AS rn                                           -- Row number column named rn
    FROM workspace.silver_schema.silver_delta_table     -- Source table alias
  ) WHERE rn = 1                                        -- Filter duplicates in this inner query
) AS source                                           -- Source dataset alias
ON target.method_id = source.method_id                -- Match target & source on method_id
AND target.time = source.time                          -- Also match on time (composite key)
WHEN MATCHED THEN                                      -- If matching row found in target
  UPDATE SET                                          -- Update existing target columns with source values
    target.sop = source.sop,
    target.meth_id = source.meth_id,
    target.temp_id = source.temp_id,
    target.report_template_version = source.report_template_version,
    target.seq_id = source.seq_id,
    target.seq_version = source.seq_version
WHEN NOT MATCHED THEN                                  -- If no matching row found in target
  INSERT (                                            -- Insert new row with source values
    time,
    sop,
    method_id,
    meth_id,
    temp_id,
    report_template_version,
    seq_id,
    seq_version
  )
  VALUES (
    source.time,
    source.sop,
    source.method_id,
    source.meth_id,
    source.temp_id,
    source.report_template_version,
    source.seq_id,
    source.seq_version
  );
